<font color="#CC3D3D"><p>
# [Competition] Building a `DNN` Model with `KerasTuner`

- `KerasTuner`는 Google에서 만든 사용하기 쉽고 확장 가능한 TensorFlow 하이퍼파라미터 최적화 프레임워크
- `Define-by-Run` 문법을 사용하여 검색 공간을 쉽게 구성한 후 최적화 알고리즘 중 하나를 사용하여 해당 모델에 가장 적합한 하이퍼 파라미터 값을 탐색
  - 사용 가능한 최적화 알고리즘: Hyperband, Bayesian Optimization, Random Search

####  전역변수 설정

In [1]:
DNN_VERSION = 2.0  # submission 화일명에 사용
SEED = 0           # random_state 지정
BATCH_SIZE = 32    # batch size 지정

In [2]:
#pip install keras-tuner --upgrade

In [3]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import random

import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
print(tf.__version__)

2.10.0


In [4]:
# 매번 모델링을 할 때마다 동일한 결과를 얻으려면 아래 코드를 실행해야 함.

def reset_seeds(seed, reset_graph_with_backend=None):
    if reset_graph_with_backend is not None:
        K = reset_graph_with_backend
        K.clear_session()
        tf.compat.v1.reset_default_graph()
        print("KERAS AND TENSORFLOW GRAPHS RESET")  # optional

    np.random.seed(seed)
    random.seed(seed)
    tf.compat.v1.set_random_seed(seed)
#    os.environ['CUDA_VISIBLE_DEVICES'] = ''  # for GPU
#    print("RANDOM SEEDS RESET")  # optional

#### (전처리된) 데이터 로딩

In [5]:
# 학습 데이터
X_train = pd.read_csv('X_train_preprocessed.csv').drop(columns='ID')
y_train = pd.read_csv('y_train.csv').Salary

# 평가 데이터
X_test = pd.read_csv('X_test_preprocessed.csv')
test_id = X_test.ID
X_test = X_test.drop(columns='ID')

In [6]:
# 학습 데이터를 다시 학습과 검증 데이터로 분할
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=SEED)

<font color="blue"><p>
#### DNN 모형 생성

<font color="blue"><p>
- 1단계: 하이퍼모델 정의하기

In [7]:
# Hyperparameter 범위지정 함수: hp.Boolean(), hp.Choice(), hp.Float(), hp.Int()

def model_fn(hp):
    inputs = keras.Input(shape=(X_train.shape[1],))
    x = inputs
    x = keras.layers.Dense(hp.Choice('units', [32,64,128,256,512]),
                           hp.Choice('activation', ['relu','elu','selu','tanh']))(x)
    if hp.Boolean("dropout"):
        x = keras.layers.Dropout(0.2)(x)
    outputs = keras.layers.Dense(1, activation='linear')(x)
    model = keras.Model(inputs, outputs)
    model.compile(loss='mse', 
                  optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), 
                  metrics=[keras.metrics.RootMeanSquaredError(name='rmse')])
    return model

<font color="blue"><p>
- 2단계: Tuner 설정 & 하이퍼튜닝 수행하기

In [8]:
# Tuner 설정
tuner = kt.Hyperband(hypermodel=model_fn,
                     objective=kt.Objective('val_rmse', direction="min"), 
                     max_epochs=10,           # 하나의 모형을 훈련시키는 최대 에포크 수
                     hyperband_iterations=1,  # 반복 횟수
                     seed=SEED,
                     overwrite=True,
                     directory='dnn_tuning')

In [9]:
%%time

# 하이퍼튜닝 수행: tuner.search()의 파라미터는 Keras의 model.fit() 파라미터와 동일
tuner.search(X_train, y_train, 
             validation_data=(X_valid, y_valid),
             batch_size=BATCH_SIZE,
             callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)], 
             verbose=1)

# 튜닝결과 요약
tuner.results_summary(num_trials=1)

Trial 30 Complete [00h 00m 07s]
val_rmse: 2896.3505859375

Best val_rmse So Far: 848.4685668945312
Total elapsed time: 00h 02m 09s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning\untitled_project
Showing 1 best trials
Trial summary
Hyperparameters:
units: 128
activation: relu
dropout: True
learning_rate: 0.01
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012
Score: 848.4685668945312
CPU times: total: 7min 5s
Wall time: 2min 9s


<font color="blue"><p>
- 3단계: 최적의 하이퍼파라미터로 모델 재학습하기

In [10]:
# 모델 재현시 Seed 고정 필요
reset_seeds(SEED)

# 베스트 모델 사용
model = tuner.get_best_models(1)[0]
# submission 화일명에 사용
val_rmse = model.evaluate(X_valid, y_valid, verbose=0)[1]

# 전체 데이터로 재학습
hist = model.fit(
    x=np.concatenate((X_train, X_valid)),
    y=np.concatenate((y_train, y_valid)),
    batch_size=BATCH_SIZE*2, # 튜닝할 때 사용한 값과 다른 값으로 설정하는 것이 좋을 수 있음 
    epochs=1,                # 작은값으로 설정
    shuffle=False,      
    verbose=2,
)

259/259 - 1s - loss: 676681.6875 - rmse: 822.6066 - 774ms/epoch - 3ms/step


<font color="blue"><p>
- 4단계: 예측하기

In [11]:
# 예측값 얻기
pred = model.predict(X_test).flatten()

# submission 화일 생성
filename = f'dnn_{DNN_VERSION}_{val_rmse:.2f}.csv'
pd.DataFrame({'ID':test_id, 'Salary':pred}).to_csv(filename, index=False)

print(f'{filename} is ready to submit.')

346/346 [==============================] - 0s 1ms/step
dnn_2.0_848.47.csv is ready to submit.


<font color="#CC3D3D"><p>
# End